# HCMUS-UNIV-WebScience-VisualizeData

## PROJECT 2: MỐI QUAN HỆ CỦA DỮ LIỆU


### I. Chuẩn bị môi trường:

In [1]:
import sqlite3
import pandas as pd
import json

In [2]:
def fetch_value(sqlCommand):
    conn = sqlite3.connect("data.db",check_same_thread=False)
    ret = pd.read_sql_query(sqlCommand, conn)
    conn.close()
    return ret

# Query dữ liệu
def query_sql(sql:str):
    listData = fetch_value(sql)
    return listData

# Lấy bài viết theo website.
def get_bai_viet(website:str):
    return query_sql(f"select * from baiviet where website='{website}'")

class Utils:
    def convertHMtoFloat(HM):
        h,m = HM.split(":")
        return int(h)+(float(m)/60)

### I. Tiền xử lý dữ liệu

 #### Yêu cầu: đưa các field về dạng chuẩn chung:

1. active_time: thời gian hoạt động, đưa về dạng sau:

```
    {
        <tên ngày trong tuần>:[ <thời gian mở cửa>,<thời gian đóng cửa> ]
    }

    *lowercase
    *thời gian đưa về float: 8h30 => 8.5
```
    
- ví dụ:

```
    {
        "thứ 2": [7,21.5],
        ...,
        "chủ nhật": [8,22.3]
    }
```

2. other_service: các dịch vụ khác, đưa về dạng sau:

```
    [
        <tên dịch vụ>,
        <tên dịch vụ>,
        ...,
    ]
    
    *lowercase
```
    
- ví dụ:

```
    [
        "có wifi"
        "giao hàng",
        ...,
    ]
```

#### I.1 Foody

In [3]:
def preprocessing_Foody():
    # Code here
    data = get_bai_viet("Foody")
    active_time = data["active_time"]
    other_service = data["other_service"]
    format_time = []
    for time in active_time:
        #time = time.open + ',' + time.close
        time = json.loads(time)
        time_format = []
        time_format.append(time['open'])
        time_format.append(time['close'])
        x = {
            "thứ 2":time_format,
            "thứ 3":time_format,
            "thứ 4":time_format,
            "thứ 5":time_format,
            "thứ 6":time_format,
            "thứ 7":time_format,
            "chủ nhật":time_format,
            }
        time = json.dumps(x)
        format_time.append(time)
    data['active_time'] = format_time
    return data

#### I.2 ShopeeFood

In [11]:
def preprocessing_ShopeeFood():
    def maping(name):
        if name=="monday":
            return "thứ 2"
        if name=="tuesday":
            return "thứ 3"
        if name=="wednesday":
            return "thứ 4"
        if name=="thursday":
            return "thứ 5"
        if name=="friday":
            return "thứ 6"
        if name=="saturday":
            return "thứ 7"
        if name=="sunday":
            return "chủ nhật"

    def convert_active_time(HM):
        pre_format = {}
        at_js = json.loads(HM)
        atl = list(at_js)
        for key in atl:
            open,close = at_js[key].split("-")
            pre_format[maping(key)] = [Utils.convertHMtoFloat(open),Utils.convertHMtoFloat(close)]
        return pre_format

    data = get_bai_viet("ShopeeFood")
    data["active_time"] = data["active_time"].apply(convert_active_time)

    return data

#### I.3 GoogleReview

In [16]:
def preprocessing_GoogleReview():
    # Code here
    def maping(name):
        if name=="Thứ Hai":
            return "thứ 2"
        if name=="Thứ Ba":
            return "thứ 3"
        if name=="Thứ Tư":
            return "thứ 4"
        if name=="Thứ Năm":
            return "thứ 5"
        if name=="Thứ Sáu":
            return "thứ 6"
        if name=="Thứ Bảy":
            return "thứ 7"
        if name=="Chủ Nhật":
            return "chủ nhật"

    def convert_active_time(HM):
        pre_format = {}
        at_js = json.loads(HM)
        atl = list(at_js)
        for key in atl:
            if (key == "open"):
                pre_format = {
                    "thứ 2":[Utils.convertHMtoFloat("00:00"),Utils.convertHMtoFloat("23:59")],
                    "thứ 3":[Utils.convertHMtoFloat("00:00"),Utils.convertHMtoFloat("23:59")],
                    "thứ 4":[Utils.convertHMtoFloat("00:00"),Utils.convertHMtoFloat("23:59")],
                    "thứ 5":[Utils.convertHMtoFloat("00:00"),Utils.convertHMtoFloat("23:59")],
                    "thứ 6":[Utils.convertHMtoFloat("00:00"),Utils.convertHMtoFloat("23:59")],
                    "thứ 7":[Utils.convertHMtoFloat("00:00"),Utils.convertHMtoFloat("23:59")],
                    "chủ nhật":[Utils.convertHMtoFloat("00:00"),Utils.convertHMtoFloat("23:59")],
                    }
            if (at_js[key] == "Mở cửa cả ngày" or at_js[key] == "Đóng cửa"):
                open = "00:00"
                close = "23:59"
            else :
                time_format = at_js[key].split("–")
                open = time_format[0]
                close = time_format[len(time_format) - 1]
            
            pre_format[maping(key)] = [Utils.convertHMtoFloat(open),Utils.convertHMtoFloat(close)]
        return pre_format

    data = get_bai_viet("GoogleReview")
    data["active_time"] = data["active_time"].apply(convert_active_time)
    other_service = data["other_service"]
    
    format_other = []
    for other in other_service:
        other = other.lower()
        format_other.append(other)
    data['other_service'] = format_other

    return data

#### I.4 Tổng hợp lại dữ liệu tiền xử lý

In [17]:
# DATA = pd.concat([preprocessing_Foody(),preprocessing_ShopeeFood(),preprocessing_GoogleReview()])
preprocessing_GoogleReview()
# DATA.describe()

['\xa0Ăn tại chỗ\xa0', '\xa0Đồ ăn mang đi\xa0', '\xa0Giao hàng']
['\xa0Ăn tại chỗ\xa0', '\xa0Đồ ăn mang đi']
['\xa0Ăn tại chỗ\xa0', '\xa0Đồ ăn mang đi\xa0', '\xa0Giao hàng']
['\xa0Ăn tại chỗ\xa0', '\xa0Đồ ăn mang đi\xa0', '\xa0Không giao hàng']
['\xa0Ăn tại chỗ\xa0', '\xa0Đồ ăn mang đi\xa0', '\xa0Giao hàng']
['\xa0Ăn tại chỗ\xa0', '\xa0Đồ ăn mang đi']
['\xa0Ăn tại chỗ\xa0', '\xa0Đồ ăn mang đi']
[]
['\xa0Ăn tại chỗ\xa0', '\xa0Đồ ăn mang đi']
['\xa0Ăn tại chỗ\xa0', '\xa0Đồ ăn mang đi\xa0', '\xa0Giao hàng']
['\xa0Ăn tại chỗ\xa0', '\xa0Nhận hàng ở lề đường\xa0', '\xa0Giao hàng gián tiếp']
['\xa0Ăn tại chỗ\xa0', '\xa0Đồ ăn mang đi']
['\xa0Ăn tại chỗ\xa0', '\xa0Đồ ăn mang đi']
['\xa0Ăn tại chỗ\xa0', '\xa0Đồ ăn mang đi']
[]
['\xa0Ăn tại chỗ\xa0', '\xa0Đồ ăn mang đi\xa0', '\xa0Giao hàng']
['\xa0Ăn tại chỗ\xa0', '\xa0Đồ ăn mang đi\xa0', '\xa0Giao hàng']
['\xa0Ăn tại chỗ\xa0', '\xa0Đồ ăn mang đi']
['\xa0Ăn tại chỗ\xa0', '\xa0Đồ ăn mang đi\xa0', '\xa0Giao hàng']
['\xa0Ăn tại chỗ\xa0', '\xa0Đồ ăn 

,website_id,website,url,full_name,phone,district,rate,rate_count,favorite,active_time,price_from,price_to,other_service
0,GoogleReview_8137268a4f6895038f034aa9d832d17e,GoogleReview,https://www.google.com/search?&tbm=lcl&q=nh%C3...,Nhà hàng Ngon,02838277131,1,8.2,6.694,None,"{'thứ 3': [8.0, 22.5], 'thứ 4': [8.0, 22.5], '...",100000.0,300000.0,"[""\u00a0\u0102n t\u1ea1i ch\u1ed7\u00a0"", ""\u0..."
1,GoogleReview_8da08dc0564f8159c3612853f46ea520,GoogleReview,https://www.google.com/search?&tbm=lcl&q=nh%C3...,Sorae Restaurant & Lounge,02838272372,1,8.8,786.000,None,"{'thứ 3': [11.5, 23.0], 'thứ 4': [11.5, 23.0],...",0.0,0.0,"[""\u00a0\u0102n t\u1ea1i ch\u1ed7\u00a0"", ""\u0..."
2,GoogleReview_60021cfaf1f7d598fbba45f955ae72d0,GoogleReview,https://www.google.com/search?&tbm=lcl&q=nh%C3...,Nhà hàng Nhật Bản,02873000801,1,8.6,28.000,None,"{'thứ 3': [11.0, 23.0], 'thứ 4': [11.0, 23.0],...",0.0,0.0,"[""\u00a0\u0102n t\u1ea1i ch\u1ed7\u00a0"", ""\u0..."
3,GoogleReview_61670308738549d4ba2eb7efd2339491,GoogleReview,https://www.google.com/search?&tbm=lcl&q=nh%C3...,Nhà Hàng Tân Hải Vân,0901301428,1,8.0,1.402,None,"{'thứ 3': [0.0, 23.983333333333334], 'thứ 4': ...",500000.0,1000000.0,"[""\u00a0\u0102n t\u1ea1i ch\u1ed7\u00a0"", ""\u0..."
4,GoogleReview_f3f6f10f7c5e3faa47d839d009d662d6,GoogleReview,https://www.google.com/search?&tbm=lcl&q=nh%C3...,Nhà Hàng Ngọc Sương Saigon,02839256939,1,8.6,409.000,None,"{'thứ 3': [10.0, 22.0], 'thứ 4': [10.0, 22.0],...",500000.0,1000000.0,"[""\u00a0\u0102n t\u1ea1i ch\u1ed7\u00a0"", ""\u0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1882,GoogleReview_d07389d8f90c1250b588aa2567568bbb,GoogleReview,https://www.google.com/search?&tbm=lcl&q=nh%C3...,Quán Ăn Gỏi Cá,,tân phú,8.2,116.000,None,"{'thứ 3': [16.5, 22.0], 'thứ 4': [16.5, 22.0],...",100000.0,300000.0,"[""\u00a0\u0102n t\u1ea1i ch\u1ed7\u00a0"", ""\u0..."
1883,GoogleReview_4eab4770bfc98f6635e3647a2a44a20f,GoogleReview,https://www.google.com/search?&tbm=lcl&q=nh%C3...,Quán Xôi TRANG,,12,10.0,2.000,None,"{'thứ 3': [5.75, 10.5], 'thứ 4': [6.0, 10.5], ...",0.0,0.0,"[""\u00a0\u0102n t\u1ea1i ch\u1ed7\u00a0"", ""\u0..."
1884,GoogleReview_612e3a63c0d2dd4df0a55218368878c0,GoogleReview,https://www.google.com/search?&tbm=lcl&q=nh%C3...,giá nhà đất quận 12,0972704833,12,10.0,15.000,None,"{'thứ 3': [0.0, 23.983333333333334], 'thứ 4': ...",0.0,0.0,[]
1885,GoogleReview_c49b1230997e2e1b5e574e0a3c1087c4,GoogleReview,https://www.google.com/search?&tbm=lcl&q=nh%C3...,The Lyan Coffee,,12,5.0,0.000,None,"{'thứ 2': [0.0, 23.983333333333334], 'thứ 3': ...",0.0,0.0,"[""\u00a0\u0102n t\u1ea1i ch\u1ed7\u00a0"", ""\u0..."


### 2. Thống kê dữ liệu

// Mô tả ...

In [ ]:
# Code here

###  Phần 3,4,5

# HCMUS-UNIV-WebScience-VisualizeData

## PROJECT 2: MỐI QUAN HỆ CỦA DỮ LIỆU
